### Libraries

In [ ]:
'generic library imports'
import pandas as pd
'machine learning library imports'
from sklearn import tree
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

### Load data

In [ ]:
data_dir = os.path.abspath('../data')

# Non-augmented dataset
df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k.csv'), low_memory=False)
# SMOTE augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE.csv'), low_memory=False)
# SMOTE-NC augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE_NC.csv'), low_memory=False)
# RealTabFormer augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_RealTabFormer.csv'), low_memory=False)
# GReaT augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_GReaT.csv'), low_memory=False)


# Test data for all datasets
df_test = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_test.csv'), low_memory=False)

### Data prepraration

In [ ]:
# Labels
y_train = df_train['Attack_type']
y_test = df_test['Attack_type']

# Features
X_train = df_train.drop(['Attack_type', 'Attack_label'], axis=1)
X_test = df_test.drop(['Attack_type', 'Attack_label'], axis=1)

# Train and test data dimensions
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')